In [ ]:
import twitter as tw
import pandas as pd
import numpy as np
import time
import MeCab
from decimal import *
tagger = MeCab.Tagger ("-Ochasen")

In [ ]:
# print([user.ScreenName for user in users])
def get_follow_users(screen_name,output_tsv_file='follow_users.tsv'):
    follow_users= [[u.name,u.screen_name,u.description,u.followers_count,u.statuses_count,u.protected] for u in api.GetFriends(screen_name=screen_name)]
    follow_users_df = pd.DataFrame(follow_users,columns=["name","screen_name","description","followers_count","statuses_count","is_protected"])
    follow_users_df.to_csv(output_tsv_file,sep='\t')

In [ ]:
def get_tweets(screen_name,num_of_tweet=100):
    return api.GetUserTimeline(screen_name=screen_name,count=num_of_tweet) #Status

In [ ]:
def get_follow_users_list(input_tsv_file="follow_users.tsv",is_exclude_protected=True):
    df = pd.read_csv(input_tsv_file,sep='\t')
    #非公開ユーザーを除外
    if is_exclude_protected:
        df = df[df["is_protected"] == 0]
    #listで返却
    return df.screen_name.values.tolist()

In [ ]:
def export_followers_list(followers,output_file_name,is_exclude_protected=True):
    output = []
    for fw in followers:
        if is_exclude_protected:
            if fw.protected == False:
                #商用ツイッター（公式と入っているユーザー）やフォローしている相手が千人を超えるユーザーは除く
                if (fw.description.find('公式') == -1) and (fw.friends_count < 1000):
                    output.append(fw.screen_name)
    pd.DataFrame(output).to_csv(output_file_name,sep='\t',index=False,header=False)

In [ ]:
def get_tweets_from_csv(input_tsv_file):
    #フォローリストからツイートを取得
    tweets = []
    f_list = pd.read_csv(input_tsv_file,sep='\t')
    for screen_name in f_list:
        tweets.append(get_tweets(screen_name))
    tw = []
    for user_tweets in tweets:
        tw.append([[t.user.name,t.user.description,t.text,t.created_at] for t in user_tweets])
    return tw

In [ ]:
# 文章（ツイート）を分かち書きして言葉と品詞の二次元配列を得る
# sentence例 'ニンジンサラダのレシピ試してみた。これは本当に美味しい'
def get_parsed_sentence(sentence):
    ret = []
    sentence = tagger.parse(sentence)
    for line in sentence.splitlines():
        feature = line.split('\t')
    #     print(feature)
        if feature[0] != 'EOS': 
            tmp = []
            tmp.append(feature[0])
            tmp.append(feature[3])
        ret.append(tmp)
    return ret

In [ ]:
# set keys
CK = ''
CS = ''
AK = ''
AS = ''
api = tw.Api(consumer_key=CK,
                      consumer_secret=CS,
                      access_token_key=AK,
                      access_token_secret=AS)

In [ ]:
#上限1000人に設定。15分に１５回アクセス制限があるため、１度に200ユーザーを５回取得するだけにとどめる
TGT_SCREEN_NAME = ''
TGT_FOLLOWERS = api.GetFollowers(screen_name=TGT_SCREEN_NAME,count=200,total_count=2000)
FOLLOWERS_FILE_NAME = "tsv/tgt_followers.tsv"
export_followers_list(TGT_FOLLOWERS,FOLLOWERS_FILE_NAME)

In [ ]:
%%time
#ツイート取得
f_df = pd.read_csv(FOLLOWERS_FILE_NAME,sep='\t')
f_list = f_df.as_matrix()
tweets = []
tw = []
#ツイート取得制限は15分で900回まで
for s_name in f_list:
    tweets = []
    tweets.append(get_tweets(s_name[0],50))
    for user_tweets in tweets:
        tw.append([[t.user.name,t.user.description,t.text,t.created_at] for t in user_tweets])
    time.sleep(0.1)

In [ ]:
#取得ツイートをtsvで出力
tw_df  = pd.DataFrame(columns=['screen_name','desciption','text','created_at'])
for t in tw:
    tmp = pd.DataFrame(t,columns=['screen_name','desciption','text','created_at'])
    tw_df  =pd.concat([tw_df,tmp])
tw_df.text = tw_df.text.str.replace('\n','')
tw_df.text = tw_df.text.str.replace('\t','')
tw_df.to_csv('tsv/tweet.tsv',sep='\t')

In [ ]:
#名詞辞書の取得
noun_dic = {}
#reference:Nozomi Kobayashi, Kentaro Inui, Yuji Matsumoto, Kenji Tateishi. Collecting Evaluative Expressions for Opinion Extraction, Journal of Natural Language Processing 12(3), 203-222, 2005.
with open("./dic/pn.csv.m3.120408.trim") as f:
    for line in f:
       (key, feeling,prop) = line.rstrip('\n').split('\t')
       noun_dic[key] = feeling
#用言辞書の取得
#reference:Masahiko Higashiyama, Kentaro Inui, Yuji Matsumoto. Learning Sentiment of Nouns from Selectional Preferences of Verbs and Adjectives, Proceedings of the 14th Annual Meeting of the Association for Natural Language Processing, pp.584-587, 2008.
dec_dic = {}
with open("./dic/wago.121808.pn") as f:
    for line in f:
        (feeling,key) = line.rstrip('\n').split('\t')
        dec_dic[key] = feeling

In [ ]:
#評価リストを作成
tw_df = pd.read_csv('tsv/tweet.tsv',sep='\t')
tw_df = tw_df[tw_df.text.notnull()]
tgts = np.c_[tw_df.screen_name.as_matrix(), tw_df.desciption.as_matrix(), tw_df.text.as_matrix()]
total = 0
eva_list = []
for tgt_sent in tgts:
    ret = get_parsed_sentence(tgt_sent[2])
    for word in ret:
        tmp = []
        if '名詞' in word[1] and word[0] in noun_dic:
            tmp.append(tgt_sent[0])
            tmp.append(tgt_sent[1])
            tmp.append(word[0])
            tmp.append(noun_dic[word[0]])
            tmp.append('名詞')
            eva_list.append(tmp)
        elif ('形容詞' or '副詞' or '動詞' in word[1]) and word[0] in dec_dic:
            tmp.append(tgt_sent[0])
            tmp.append(tgt_sent[1])
            tmp.append(word[0])
            tmp.append(dec_dic[word[0]])
            tmp.append('用言')
            eva_list.append(tmp)
df = pd.DataFrame(eva_list,columns=['screen_name','description','word','feeling','type'])

In [ ]:
df['positive_flg'] = ((df['feeling'].str.contains('ポジ')) | (df['feeling'].str.contains('p'))).astype(int)
df['negative_flg'] = ((df['feeling'].str.contains('ネガ')) | (df['feeling'].str.contains('n'))).astype(int)
df['eva'] = df['positive_flg'] + df['negative_flg'] * -1

In [ ]:
#分かち書きスコアつきツイート出力
df.to_csv('tsv/evaluated_tweet.tsv',sep='\t')

In [ ]:
#ユーザースコア
grp_df = df.groupby('screen_name')['eva'].mean()
grp_df.to_csv('tsv/grp_evaluated_tweet.tsv',sep='\t')

In [ ]:
eva_df = pd.read_csv("tsv/evaluated_tweet.tsv",sep='\t')
grp_df = pd.read_csv("tsv/grp_evaluated_tweet.tsv",sep='\t',names=("screen_name","score"))

In [ ]:
eva_score = eva_df.groupby('screen_name',as_index=False)['eva'].mean()
eva_score.rename(columns={'eva':'score'})
eva_word_count = eva_df.groupby(['screen_name','word'],as_index=False).size()

In [ ]:
eva_word_count = eva_df.groupby(['screen_name','word','positive_flg','negative_flg'],as_index=False).count()
eva_word_count.rename(columns={'Unnamed: 0':'cnt'})

In [ ]:
#ユーザーごとのスコアをマージ
mrg_df = pd.merge(eva_df, grp_df,on="screen_name",how="left",left_index=False,right_index=False,sort=True)
mrg_df = pd.merge(mrg_df, eva_score,on="screen_name",how="left",left_index=False,right_index=False,sort=True)

In [ ]:
mrg_df.to_csv("tsv/merge_eva_tweet.tsv",sep='\t')
eva_word_count.to_csv("tsv/eva_word_count.tsv",sep='\t')
